In [1]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
from utils.ScoreFunction import score_function

random_seed = 42
np.random.seed(random_seed)

## Пример работы скоринговой функции

In [2]:
y_true = [1, 0, 0, 1, 1, 1, 0, 0, 0, 0]
y_pred = [1, 1, 1, 0, 0, 0, 0, 0, 0, 0]
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

In [3]:
(tp,fp,fn,tn)

(1, 2, 3, 4)

In [4]:
score_function(tp,fp,fn,tn)

-60

## Импорт данных

In [235]:
data = pd.read_csv("../data/train.csv",sep='|')
X = data.drop(["fraud"],axis=1)
y = data["fraud"]

X['totalScanned'] = X['scannedLineItemsPerSecond']*X['totalScanTimeInSeconds']
X['avgTimePerScan'] = 1/X['scannedLineItemsPerSecond']
X['avgValuePerScan'] = X['avgTimePerScan']*X['valuePerSecond']
X['withoutRegisPerPosition'] = X['scansWithoutRegistration']*X['totalScanned']
X['quantityModsPerPosition'] = X['quantityModifications']/X['totalScanned']

#X['lineItemVoidsPerPosition'] = X['lineItemVoids']/X['totalScanned'] #
#X['lineItemVoidsPerTotal'] = X['lineItemVoids']/X['grandTotal'] #
#X['withoutRegistrationPerTotal'] = X['scansWithoutRegistration']/X['grandTotal'] #
#X['quantiModsPerTotal'] = X['quantityModifications']/X['grandTotal'] #
#X['lineItemVoidsPerTime'] = X['lineItemVoids']/X['totalScanTimeInSeconds'] #
#X['withoutRegistrationPerTime'] = X['scansWithoutRegistration']/X['totalScanTimeInSeconds'] #
#X['quantiModesPerTime'] = X['quantityModifications']/X['totalScanTimeInSeconds'] #

In [236]:
X = np.array(X)
y = np.array(y)

## Создание и обучение линейной регрессии, валидация

In [237]:
skf = StratifiedKFold(n_splits=5)
results = []

In [238]:
for train_index, test_index in skf.split(X, y):
    clf = LogisticRegression(random_state=random_seed)
    clf.fit(X[train_index], y[train_index])
    y_pred = clf.predict(X[test_index])
    tn, fp, fn, tp = confusion_matrix(y[test_index], y_pred).ravel()
    results.append(score_function(tp,fp,fn,tn))
    print(score_function(tp,fp,fn,tn),f1_score(y[test_index], y_pred))

25 0.8780487804878048
60 0.9268292682926829
80 0.9767441860465117
35 0.9047619047619048
60 0.888888888888889


In [239]:
np.array(results).mean()

52.0